In [137]:
import re
import pandas as pd
import pickle
import os

In [138]:
articles_metadata = pd.read_json("../data/article_metadata.json")
articles_metadata

,pop_id,year,authors,journal,title,type,filepath
0,0,2011,Bos et al.,Political Communication,How the Media Shape Perceptions of Right-Wing ...,article,Political Communication/2011 - Bos et al. - Ho...
1,1,2019,Blassnig et al.,Political Communication,Hitting a Nerve: Populist News Articles Lead t...,article,Political Communication/2019 - Blassing et al....
2,2,2017,Caramani,American Political Science Review,Will vs. Reason: The Populist and Technocratic...,article,American Political Science Review/2017 - Caram...
3,3,2020,"Wuttke,Schimpf, Schoen",American Political Science Review,When the Whole Is Greater than the Sum of Its ...,article,American Political Science Review/2020 - Wuttk...
4,4,2014,Treib,Journal of European Public Policy,"The voter says no, but nobody listens: causes ...",article,Journal of European Public Policy/2014 - Treib...
...,...,...,...,...,...,...,...
536,536,2018,Grzymala-Busse,Current History,Poland’s Path to Illiberalism,article,Current History/2018 - Grzymala-Busse - Poland...
537,537,2017,Eichengreen,Current History,The Populist Turn in American Politics,article,Current History/2017 - Eichengreen - The Popul...
538,538,2014,Mudde,Current History,The Far Right and the European Elections,article,Current History/2014 - Mudde - The Far Right a...
539,539,2020,Cano,European History Quarterly,Printing the People,article,European History Quarterly/2020 - Cano - Print...


In [139]:
articles_metadata[articles_metadata["pop_id"]==17]

,pop_id,year,authors,journal,title,type,filepath
17,17,2018,Rooduijn,European Political Science Review,What unites the voter bases of populist partie...,article,European Political Science Review/2018 - Roodu...


In [143]:
# create folder for filtered texts
!rm -rf ../data/large_data/articles_filteredtexts
!mkdir ../data/large_data/articles_filteredtexts

In [144]:
path = "../data/large_data/articles_filteredtexts/"

In [145]:
filteredtext = "In the middle of 2011, both Turkey and Thailand held national elections,124 While Turkish premier Recep Tayyip Erdo¢gan claimed his third popular mandate, his self-exiled Thai doppelgänger Thaksin Shinawatra—whose parties have now won five general elections since 2001—saw his younger sister and stand-in Yingluck Shinawatra assume the premiership."

In [146]:
re.sub("([\.|\:|\?\,])(\d+)", r"\1", filteredtext)

'In the middle of 2011, both Turkey and Thailand held national elections, While Turkish premier Recep Tayyip Erdo¢gan claimed his third popular mandate, his self-exiled Thai doppelgänger Thaksin Shinawatra—whose parties have now won five general elections since 2001—saw his younger sister and stand-in Yingluck Shinawatra assume the premiership.'

In [147]:
spliters = ["References\n",
            "Bibliography\n",
            "Notes\n",
            "Acknowledgments\n"]
def text_cleaning(filteredtext):
    filteredtext_len = len(filteredtext)
    usable_spliters = []
    for spliter in spliters + [spliter.upper() for spliter in spliters]:
        if spliter in filteredtext:
            keeping_len = len(filteredtext.partition(spliter)[0])
            if keeping_len > filteredtext_len / 2:
                usable_spliters.append((keeping_len, spliter))
    if bool(usable_spliters):
        keeping_index = min(usable_spliters)[0]
        filteredtext = filteredtext[:keeping_index]
    filteredtext = filteredtext.replace("-\n", "")
    filteredtext = re.sub("([\.|\:|\?\,])(\d+)", r"\1", filteredtext)
    return filteredtext

In [148]:
pop_id = 75
pct = 3
filename = "textblocks_pop_id_{}.pickle".format(str(pop_id))
pages_back = pickle.load(open("../data/large_data/articles_textblocks/" + filename, "rb"))
rawtext = ""
for p in pages_back[1:]:
    for tb in p:
        rawtext += tb[4]
rawtext_per_page = len(rawtext) / len(pages_back[1:])
filteredtext = ""
end_here = False
for p in pages_back:
    accepted_tbs = []
    last_included = False
    for tb in p:
        tb_text = tb[4]
        if len(re.sub("\d+|\W", "", tb_text)) / len(tb_text) > 0.6:
            # main filtering condition:
            #if tb_text.count("\n") > 2:
            if last_included == False:
                if len(tb_text) >= rawtext_per_page * pct / 100:
                    accepted_tbs.append(tb_text)
                    last_included = True
            else:
                accepted_tbs.append(tb_text)
            if tb_text in spliters + [spliter.upper() for spliter in spliters]:
                end_here = True
                break
    accepted_tbs_reversed = []
    last_included = False
    for tb_text in reversed(accepted_tbs):
        if last_included == False:
            if len(tb_text) >= rawtext_per_page * pct / 100:
                accepted_tbs_reversed.append(tb_text)
                last_included = True
        else:
            accepted_tbs_reversed.append(tb_text)
    accepted_tbs_reversed = [tb for tb in accepted_tbs_reversed if tb[0] != "<"]
    filteredtext += " ".join(reversed(accepted_tbs_reversed))
    if end_here == True:
        break
filteredtext = text_cleaning(filteredtext)


In [149]:
print(filteredtext)

[ This content has been declared free to read by the pubisher during the COVID-19 pandemic. ]
Laurence Whitehead is senior research fellow in politics at Nuffield 
College, Oxford. He is coeditor (with Saskia Ruth-Lovell and Yanina 
Welp) of Let the People Rule? Direct Democracy in the Twenty-First 
Century (2017).
 The United Kingdom’s 12 December 2019 parliamentary election was 
a watershed. With 43. percent of the vote on a 67. percent turnout, 
Prime Minister Boris Johnson’s Conservative Party secured a majority 
of 80-plus seats in Parliament’s 650-member House of Commons—the 
largest in two decades. The opposition Labour Party, with 32. percent 
of the vote, suffered defeat on a scale comparable only to that of 1983, 
when it won not quite 28 percent. In terms of seats, Labour’s harvest 
was even worse—it lost 59 to bring its total down to 203, the lowest 
level since 1935. 
The Conservatives gained 47 seats for a total of 365, while the Scottish National Party (SNP) came in thir

In [150]:
def get_filteredtext(pop_id, pct=1, tofile=True):
    filename = "textblocks_pop_id_{}.pickle".format(str(pop_id))
    pages_back = pickle.load(open("../data/large_data/articles_textblocks/" + filename, "rb"))
    rawtext = ""
    for p in pages_back:
        for tb in p:
            rawtext += tb[4]
    rawtext_per_page = len(rawtext) / len(pages_back)
    filteredtext = ""
    end_here = False
    for p in pages_back:
        accepted_tbs = []
        last_included = False
        for tb in p:
            tb_text = tb[4]
            if len(re.sub("\d+|\W", "", tb_text)) / len(tb_text) > 0.5:
                # main filtering condition:
                #if tb_text.count("\n") > 2:
                if last_included == False:
                    if len(tb_text) >= rawtext_per_page * pct / 100:
                        accepted_tbs.append(tb_text)
                        last_included = True
                else:
                    accepted_tbs.append(tb_text)
                if tb_text in spliters + [spliter.upper() for spliter in spliters]:
                    end_here = True
                    break
        accepted_tbs_reversed = []
        last_included = False
        for tb_text in reversed(accepted_tbs):
            if last_included == False:
                if len(tb_text) >= rawtext_per_page * pct / 100:
                    accepted_tbs_reversed.append(tb_text)
                    last_included = True
            else:
                accepted_tbs_reversed.append(tb_text)
        accepted_tbs_reversed = [tb for tb in accepted_tbs_reversed if tb[0] != "<"]
        filteredtext += " ".join(reversed(accepted_tbs_reversed))
        if end_here == True:
            break
    filteredtext = text_cleaning(filteredtext)
    if tofile:
        newfile = "filteredtext_pop_id_{}.txt".format(str(pop_id))
        with open(path + newfile, "w") as f:
            f.write(filteredtext)
    else:
        return filteredtext

In [151]:
print(get_filteredtext(398, tofile=False))

How should we analyze the connections between
populism and nationalism: A response to Rogers
Brubaker
 Benjamin De Cleen1
|
Yannis Stavrakakis2
 1Department of Communication Studies, Vrije Universiteit Brussel, Brussel, Belgium
 2Department of Political Science, Aristotle University of Thessaloniki, Thessaloniki, Greece
 Correspondence
Benjamin De Cleen, Department of
Communication Studies, Vrije Universiteit
Brussel, Brussel, Belgium.
Email: benjamin.de.cleen@vub.be
 Abstract
 In a recent article in this journal, Rogers Brubaker formu lates a critique of the distinction we make in our work
 between populism and nationalism, and further develops his
 own, thicker conceptualization of populism, which inte grates the nationalist dimension without however totally
 conflating populism and nationalism. In this article, we
 briefly respond to the critique of our work, further clarifying
 and refining our plea for clearly distinct conceptualizations
 of populism and nationalism in dialogue wi

In [153]:
path = "../data/large_data/articles_filteredtexts/"
for id in articles_metadata["pop_id"]:
    get_filteredtext(id)